In [19]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
import json
import datetime

from pmdarima.arima import ADFTest
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import auto_arima
    
    
plt.style.use('fivethirtyeight')

# criando a pasta de registro de calibração
if not os.path.isdir("files"):
    os.mkdir("files")

In [20]:
def get_finance_data(ticker, period='1y', interval='1d'):
    tkr = yf.Ticker(ticker)
    df = tkr.history(period=period, interval=interval)
    return df

def test_unit_root(s):
    adf = adfuller(s)
    return adf[0] > adf[4]['1%']

def stationary_test(s):
    adf_test = ADFTest(alpha=0.01)
    return not adf_test.should_diff(s)[1]

def get_arima_model(s):
    is_stat = stationary_test(s)
    is_seasonal = test_unit_root(s) 
    arima_model = auto_arima(s, stationary=is_stat, start_p=0, d=1, start_q=0, max_p=5, max_d=5, max_q=5, start_P=0, D=1, 
                             start_Q=1, max_P=5, max_D=5, max_Q=5, m=12, seasonal=is_seasonal, error_action='warn', trace=True, 
                             suppress_warnings=True, stepwise=False, random_state=20, n_fits=50, n_jobs=-1)
    return arima_model

In [44]:
def run_arima_coach(yticker_list):
    n_steps = 2
    for yticker in yticker_list:
        ticker = yticker
        print("capturando dados do yFinance: "+yticker)
        data = get_finance_data(yticker)
        train = data['Low'][:len(data)-n_steps+1]
        test = data['Low'][-n_steps:]
        if os.path.isfile("files/db.json"):
            with open('files/db.json','r+') as jfile:
                jdata = json.load(jfile)
                if ticker not in jdata.keys():
                    print("ticker "+ticker+" não encontrado")
                    jdata[ticker]={}
                    jdata[ticker].update({"yticker":yticker})
                    jfile.seek(0)
                    json.dump(jdata, jfile)
                else:
                    print("dados de "+ticker+" já capturados")

        my_arima = []
        if "ARIMA" not in jdata[ticker]:
            print("rodando auto arima para "+ticker)
            arima_model = get_arima_model(train)
            my_arima.append(arima_model.order[0])
            my_arima.append(arima_model.order[1])
            my_arima.append(arima_model.order[2])
            my_arima.append(arima_model.seasonal_order[0])
            my_arima.append(arima_model.seasonal_order[1])
            my_arima.append(arima_model.seasonal_order[2])
            my_arima.append(arima_model.seasonal_order[3])    
            with open('files/db.json','r+') as jfile:
                hoje = datetime.datetime.now().strftime("%d-%m-%Y")
                jdata[ticker].update({"ARIMA":{"parametros":my_arima,"train_date":hoje}})
                jfile.seek(0)
                json.dump(jdata, jfile)
        
        jfile.close()
            
        #temp
        print(jdata)

In [46]:
teste = ["VILG11.SA","RNDP11.SA"]

run_arima_coach(teste)

capturando dados do yFinance: VILG11.SA
dados de VILG11.SA já capturados
{'BBFI11B.SA': {'yticker': 'BBFI11B.SA', 'ARIMA': {'parametros': [0, 1, 0, 4, 1, 0, 12], 'train_date': '01-01-2021'}}, 'VILG11.SA': {'yticker': 'VILG11.SA', 'ARIMA': {'parametros': [], 'train_date': '01-01-2021'}}, 'RNDP11.SA': {'yticker': 'RNDP11.SA', 'ARIMA': {'parametros': [0, 1, 1, 4, 1, 0, 12], 'train_date': '01-01-2021'}}}
capturando dados do yFinance: RNDP11.SA
dados de RNDP11.SA já capturados
{'BBFI11B.SA': {'yticker': 'BBFI11B.SA', 'ARIMA': {'parametros': [0, 1, 0, 4, 1, 0, 12], 'train_date': '01-01-2021'}}, 'VILG11.SA': {'yticker': 'VILG11.SA', 'ARIMA': {'parametros': [], 'train_date': '01-01-2021'}}, 'RNDP11.SA': {'yticker': 'RNDP11.SA', 'ARIMA': {'parametros': [0, 1, 1, 4, 1, 0, 12], 'train_date': '01-01-2021'}}}


NameError: name 'jdata' is not defined